# Handling Missing Data

Let us assume that the following model generates the data
$$
\begin{align*}
    {x}_t &= {x}_{t-1} + c \\
    {y}_t &\sim \mathcal{N}\left({x}_{t}, p \right) 
\end{align*}
$$
with prior $${x}_0 \sim \mathcal{N}({m_{{x}_0}}, {v_{{x}_0}})$$.

Suppose that our measurement device fails to acquire data from time to time.  In this case, instead of scalar observation $\hat{y}_t \in \mathrm{R}$ we sometimes will catch `missing` observations.

In [10]:
using Rocket
using ReactiveMP
using GraphPPL
using BenchmarkTools
using Distributions

In [26]:
@model function smoothing(n, x0, P::ConstVariable)
    
    x_prior ~ NormalMeanVariance(mean(x0), cov(x0)) 

    x = randomvar(n)
    y = datavar(Float64, n) where { allow_missing = true }
    c = constvar(1.0)

    x_prev = x_prior

    for i in 1:n
        x[i] ~ x_prev + c
        y[i] ~ NormalMeanVariance(x[i], P)
        
        x_prev = x[i]
    end

    return x, y
end

In [27]:
@rule NormalMeanVariance(:μ, Marginalisation) (m_out::Any, m_v::Missing) = missing
@rule NormalMeanVariance(:μ, Marginalisation) (m_out::Missing, m_v::Any) = missing

@rule typeof(+)(:in1, Marginalisation) (m_out::Missing, m_in2::Any) = missing
@rule typeof(+)(:in1, Marginalisation) (m_out::Any, m_in2::Missing) = missing

In [29]:
P = 1.0

n = 500
data = convert(Vector{Union{Float64, Missing}}, collect(1:n) + rand(Normal(0.0, sqrt(P)), n));

for index in map(d -> rem(abs(d), n), rand(Int, Int(n / 2)))
    data[index] = missing
end

In [32]:
x0_prior = NormalMeanVariance(0.0, 1000.0)

result = inference(model=Model(smoothing, n, x0_prior, P), data=(y=data,), returnvars=(x=KeepLast(),))



LoadError: MethodError: no method matching iterate(::Missing)
[0mClosest candidates are:
[0m  iterate([91m::Union{LinRange, StepRangeLen}[39m) at range.jl:664
[0m  iterate([91m::Union{LinRange, StepRangeLen}[39m, [91m::Int64[39m) at range.jl:664
[0m  iterate([91m::T[39m) where T<:Union{Base.KeySet{var"#s79", var"#s78"} where {var"#s79", var"#s78"<:Dict}, Base.ValueIterator{var"#s77"} where var"#s77"<:Dict} at dict.jl:693
[0m  ...